#3D cloud field 

This folder conatains the 3D cloud field from which slice was taken for showing a case of the single-scattering adjoint in 2D.  The purpose of this notebook will be to plot the 3D cloud field. 


In [1]:
%gui wx

# Imports
import numpy as np
import scipy as sp
from enthought.mayavi import mlab

import pandas as pd



In [2]:
# Utility functions

def print_head(fname, Nlines=10, indent="\t"):
    "Print the head of the file."
    
    # Print a message and then the first N lines
    print("Showing head: {}".format(fname))
    for i, line in zip(range(Nlines), open(fname, 'r')):
        print(indent + line.strip())
        
    print('\n')
    

In [3]:
ls

cloud-field.ipynb  cloud-field-slice.jpg  les0822nh15t13.lwc~
cloud-field.jpg    les0822nh15t13.lwc


# Looking at the data files

In [4]:
# The log file
flwc = 'les0822nh15t13.lwc'
print_head(flwc, Nlines=15)

Showing head: les0822nh15t13.lwc
	! 2  parameter LWC file
	! NX  NY  NZ
	! 320 320 101
	!
	! DX     DY
	! 0.0625 0.0625
	!
	! Zlevels
	! -0.0087  0.0095  0.0315  0.0574  0.0878  0.1245  0.1650  0.2050  0.2450  0.2850  0.3250  0.3650  0.4050  0.4450  0.4850  0.5250  0.5650  0.6050  0.6450  0.6850  0.7250  0.7650  0.8050  0.8450  0.8850  0.9250  0.9650  1.0050  1.0450  1.0850  1.1250  1.1650  1.2050  1.2450  1.2850  1.3250  1.3650  1.4050  1.4450  1.4850  1.5250  1.5650  1.6050  1.6450  1.6855  1.7222  1.7526  1.7780  1.7992  1.8190  1.8390  1.8590  1.8790  1.8990  1.9190  1.9390  1.9590  1.9790  1.9990  2.0190  2.0388  2.0600  2.0854  2.1158  2.1525  2.1930  2.2330  2.2730  2.3130  2.3530  2.3930  2.4330  2.4730  2.5130  2.5530  2.5930  2.6330  2.6730  2.7130  2.7530  2.7930  2.8330  2.8730  2.9130  2.9530  2.9930  3.0330  3.0730  3.1130  3.1530  3.1930  3.2330  3.2730  3.3130  3.3530  3.3930  3.4330  3.4730  3.5130  3.5530  3.5930
	!
	! Temperatures
	! 298.15 297.97 297.73 297.48 297.1

In [5]:
# Read the grid parameters
header_lines = []
for line in open(flwc, 'r'):
    line = line.strip()
    if line[0] == '!': 
        header_lines.append(line)
    else:
        break
        
lwc_frame = pd.read_csv(flwc, quoting=3, delim_whitespace=True, skiprows=13)
print(lwc_frame[:5])        



   IX  IY  IZ     LWC  Reff
0   1   1  47  0.0100  3.82
1   1   1  48  0.0263  5.27
2   1   1  49  0.0300  5.51
3   1   1  50  0.0186  4.69
4   1  53  66  0.0058  3.18


In [6]:
# Get the variables 
NX = 320
DX = 0.0625
X = np.arange(NX) * DX
NY = 320
DY = 0.0625
Y = np.arange(NY) * DY
NZ = 101
Z = np.array("-0.0087  0.0095  0.0315  0.0574  0.0878  0.1245  0.1650  0.2050  0.2450  0.2850  0.3250  0.3650  0.4050  0.4450  0.4850  0.5250  0.5650  0.6050  0.6450  0.6850  0.7250  0.7650  0.8050  0.8450  0.8850  0.9250  0.9650  1.0050  1.0450  1.0850  1.1250  1.1650  1.2050  1.2450  1.2850  1.3250  1.3650  1.4050  1.4450  1.4850  1.5250  1.5650  1.6050  1.6450  1.6855  1.7222  1.7526  1.7780  1.7992  1.8190  1.8390  1.8590  1.8790  1.8990  1.9190  1.9390  1.9590  1.9790  1.9990  2.0190  2.0388  2.0600  2.0854  2.1158  2.1525  2.1930  2.2330  2.2730  2.3130  2.3530  2.3930  2.4330  2.4730  2.5130  2.5530  2.5930  2.6330  2.6730  2.7130  2.7530  2.7930  2.8330  2.8730  2.9130  2.9530  2.9930  3.0330  3.0730  3.1130  3.1530  3.1930  3.2330  3.2730  3.3130  3.3530  3.3930  3.4330  3.4730  3.5130  3.5530  3.5930".split(), dtype="f8")
assert NZ == Z.size


# Experimenting to make a plot
ix = sp.array(lwc_frame['IX']-1, dtype=int)
iy = sp.array(lwc_frame['IY']-1, dtype=int)
iz = sp.array(lwc_frame['IZ']-1, dtype=int)
xx = X[ix]
yy = Y[iy]
zz = Z[iz]

#Make the liquid water content
lwc = np.array(lwc_frame['LWC'], dtype='f8')

# Make Structured data
xgrid, ygrid, zgrid = sp.meshgrid(X, Y, Z, indexing='ij')
lwcgrid = xgrid*0
for _ix, _iy, _iz, _lwc in zip(ix, iy, iz, lwc):
    lwcgrid[_ix, _iy, _iz] = _lwc
    
# Make a modis grid
spacing_modis = .25
_xmod = np.linspace(0,20, 20 / spacing_modis)
_zmod = np.linspace(0,3, 3/spacing_modis) #sp.array([Z.min(), Z.max()])
xmod, ymod, zmod = sp.meshgrid(_xmod, _xmod, _zmod, indexing='ij')



In [7]:
lwcgrid.min()
Y.max()

19.9375

In [8]:
Y[135]

8.4375

In [18]:
mlab.figure(1, bgcolor=(1,1,1), size=(2040,1440))
mlab.clf()



points = mlab.points3d(xx, yy, zz, lwc, color=(.9,.9,.9), opacity=.5, resolution=8, mask_points=12, scale_factor=.3)
#sufrace_grid_modis = mlab.points3d(xmod[..., 1], ymod[..., 1], zmod[...,1], mode='2dcross', color=(0.1, 0.1, 0.1), opacity=.1)

#mlab.contour3d(xgrid, ygrid, zgrid, lwcgrid, contours=3)


sufrace_grid_modis = mlab.mesh(xmod[..., 0], ymod[..., 0], zmod[...,0], representation='wireframe', 
                               color=(0.3, 0.3, 0.5))
back1 = mlab.mesh(xmod[0,...], ymod[0,...], zmod[0,...], representation='wireframe', 
                  color=(0.3, 0.3, .5),)
back2 = mlab.mesh(xmod[:,0,:], ymod[:,0,:], zmod[:,0,:], representation='wireframe', 
                  color=(0.3, 0.3, .5),)
#source = mlab.pipeline.scalar_field(xgrid, ygrid, zgrid, lwcgrid)
#vol = mlab.pipeline.volume(source, 
#                           vmin = lwc.min() + .5 * (lwc.max()-lwc.min()), 
#                           vmax = lwc.min() + .9 * (lwc.max()-lwc.min()))

mlab.savefig("cloud-field.jpg", magnification=4)


mlab.mesh(xgrid[:,135,:], ygrid[:,135,:], zgrid[:,135,:], scalars=lwcgrid[:,135,:], opacity=.5, colormap='gray')
mlab.savefig("cloud-field-with-slice.jpg", magnification=4)

mlab.show()
#points = mlab.points3d(xx, yy, zz, lwc, mode='point')